In [1]:
from __future__ import division, absolute_import

import os
import sys
import random

#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
#os.environ["CUDA_VISIBLE_DEVICES"] = ""

import h5py
import pickle
import numpy as np
import keras
import pandas as pd
import random

from glob import glob
from keras.models import load_model
from keras import models
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import datetime
from plotnine import *

#root
absPath = '/home/angela/padding_EBI/'
sys.path.insert(0, absPath)

from src.Target import Target
from src.processing_activations import *

#para no imprimir warnings chungos
import warnings 
warnings.simplefilter("ignore")

np.random.seed(1)
random.seed(1)
%matplotlib inline

Using TensorFlow backend.


In [2]:
list_paddings = ['ext_padding', 'mid_padding', 'post_padding', 'pre_padding', 
                 'rnd_padding', 'strf_padding',  "zoom_padding"]

nfolds = 10
folder = 'EC_number/archaea/1conv/64filts_size5/'
nshape = (1, 1000, 26)
nlayer = 2

In [3]:
#coger al azar 7 enzimas y 7 no enzimas para comparar
prots_pca_task1 = []
list_indices_enzymes_task1 = []
for enz_label in [0,1]:
    indices_enzymes_task1 = find_prot_labeled(folder, enz_label, 7, "task1", "labels_task1")
    print(indices_enzymes_task1, enz_label)
    list_indices_enzymes_task1.append((indices_enzymes_task1, enz_label))
    for idx,i in enumerate(indices_enzymes_task1):
        for padding in list_paddings:
            seq = load_prot_labeled(folder, padding, nshape, enz_label, i)
            prots_pca_task1.append((seq, enz_label, padding))
            #print(enz_label, "prot", idx)

[15066, 12675, 2534, 13134, 18931, 454, 3706] 0
[6586, 10257, 6480, 11790, 11511, 16316, 9507] 1


# Task 1

In [4]:
task = 'task1/'

In [5]:
prots_pca[0]

NameError: name 'prots_pca' is not defined

In [6]:
#hay que definir que vaya prot por prot y fold por fold

In [ ]:
#Define list of dataframes
lista_dfs = []
for tup in prots_pca_task1:
    model_type = tup[2]
    enz_type = tup[1]
    prot = tup[0]
    for fold in range(nfolds):
        #print("Model type: ", model_type, "enzyme type", enz_type)
        model = search_model(folder, str(fold), task, model_type)
        #writing log file 
        print('Model type: %s \n' % model_type)
        print('Enzyme type: %s \n' % enz_type)
        formatted_time = datetime.datetime.now()
        print('Starting time: %s \n' % formatted_time)
        activations = obtain_activations(model, nlayer, prot)
        act_df = activations_to_df(activations, model_type, fold)
        act_df["enzyme_type"] = tup[1]
        formatted_endtime = datetime.datetime.now()
        print('Finishing time: %s \n' % formatted_endtime)
        lista_dfs.append(act_df)

Model type: ext_padding 

Enzyme type: 0 

Starting time: 2019-10-31 17:03:46.958523 

Finishing time: 2019-10-31 17:03:52.526882 

Model type: ext_padding 

Enzyme type: 0 

Starting time: 2019-10-31 17:03:52.542160 

Finishing time: 2019-10-31 17:03:53.580872 

Model type: ext_padding 

Enzyme type: 0 

Starting time: 2019-10-31 17:03:53.581765 

Finishing time: 2019-10-31 17:03:54.643698 

Model type: ext_padding 

Enzyme type: 0 

Starting time: 2019-10-31 17:03:54.647334 

Finishing time: 2019-10-31 17:03:55.921764 

Model type: ext_padding 

Enzyme type: 0 

Starting time: 2019-10-31 17:03:55.922457 

Finishing time: 2019-10-31 17:03:57.344865 

Model type: ext_padding 

Enzyme type: 0 

Starting time: 2019-10-31 17:03:57.345812 

Finishing time: 2019-10-31 17:03:58.811928 

Model type: ext_padding 

Enzyme type: 0 

Starting time: 2019-10-31 17:03:58.812812 

Finishing time: 2019-10-31 17:04:00.526045 

Model type: ext_padding 

Enzyme type: 0 

Starting time: 2019-10-31 17:04:0

Finishing time: 2019-10-31 17:10:01.931582 

Model type: zoom_padding 

Enzyme type: 0 

Starting time: 2019-10-31 17:10:01.932142 

Finishing time: 2019-10-31 17:10:13.673550 

Model type: zoom_padding 

Enzyme type: 0 

Starting time: 2019-10-31 17:10:13.674120 

Finishing time: 2019-10-31 17:10:25.596121 

Model type: zoom_padding 

Enzyme type: 0 

Starting time: 2019-10-31 17:10:25.596808 

Finishing time: 2019-10-31 17:10:38.318628 

Model type: zoom_padding 

Enzyme type: 0 

Starting time: 2019-10-31 17:10:38.319186 

Finishing time: 2019-10-31 17:10:50.670286 

Model type: zoom_padding 

Enzyme type: 0 

Starting time: 2019-10-31 17:10:50.679704 

Finishing time: 2019-10-31 17:11:03.287540 

Model type: zoom_padding 

Enzyme type: 0 

Starting time: 2019-10-31 17:11:03.288428 

Finishing time: 2019-10-31 17:11:16.105763 

Model type: zoom_padding 

Enzyme type: 0 

Starting time: 2019-10-31 17:11:16.128949 

Finishing time: 2019-10-31 17:11:29.166463 

Model type: ext_padding 

Finishing time: 2019-10-31 17:29:16.811635 

Model type: strf_padding 

Enzyme type: 0 

Starting time: 2019-10-31 17:29:16.812456 

Finishing time: 2019-10-31 17:29:42.979497 

Model type: strf_padding 

Enzyme type: 0 

Starting time: 2019-10-31 17:29:42.980292 

Finishing time: 2019-10-31 17:30:09.526323 

Model type: strf_padding 

Enzyme type: 0 

Starting time: 2019-10-31 17:30:09.527116 

Finishing time: 2019-10-31 17:30:37.278373 

Model type: strf_padding 

Enzyme type: 0 

Starting time: 2019-10-31 17:30:37.279187 

Finishing time: 2019-10-31 17:31:04.409305 

Model type: strf_padding 

Enzyme type: 0 

Starting time: 2019-10-31 17:31:04.410158 

Finishing time: 2019-10-31 17:31:31.798866 

Model type: zoom_padding 

Enzyme type: 0 

Starting time: 2019-10-31 17:31:31.799665 

Finishing time: 2019-10-31 17:31:59.474308 

Model type: zoom_padding 

Enzyme type: 0 

Starting time: 2019-10-31 17:31:59.474885 

Finishing time: 2019-10-31 17:32:27.382223 

Model type: zoom_padding

In [ ]:
# ha tardado un huevo en ejecutarse asi que deberia guardarlo
filename = "activations_enzymeType.pickle"
pickle_path = "".join((absPath, "data/results/", folder, task, filename))
with open(pickle_path, 'wb') as handle:
    pickle.dump(lista_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

#loading it
#with open(filename, 'rb') as handle:
#    lista_dfs = pickle.load(handle)

In [ ]:
1+1

In [ ]:
#TASK2

prots_pca = []
list_indices_enzymes = []
for enz_label in range(1,8):
    indices_enzymes = find_prot_labeled(folder, enz_label)
    print(indices_enzymes, enz_label)
    list_indices_enzymes.append((indices_enzymes, enz_label))
    for idx,i in enumerate(indices_enzymes):
        for padding in list_paddings:
            seq = load_prot_labeled(folder, padding, nshape, enz_label, i)
            prots_pca.append((seq,enz_label, padding))
            #print(enz_label, "prot", idx)

In [ ]:
lista_dfs[0].columns

In [ ]:
lista_dfs[0].shape

In [ ]:
final_df, pca = train_pca(lista_dfs, 2)

In [ ]:
final_df.shape

In [ ]:
plot_pca(final_df, pca, "padding", list_paddings)

In [ ]:
plot_pca_per_padding(final_df, pca)

In [ ]:
density_facet(final_df, pca)

In [ ]:
#cambiar respecto a la nueva funcion de train_pca
df_joint = pd.concat(lista_dfs, ignore_index=True)
x = df_joint.drop(['order', 'padding', 'fold', "enzyme_type",
                       #, 'label'
                      ], axis=1).values
x = StandardScaler().fit_transform(x)
pca = PCA(n_components=2)
pca.fit(x)
principalComponents = pca.fit_transform(x)

In [ ]:
myplot(principalComponents, np.transpose(pca.components_))

In [ ]:
# guardamos final_df en un csv
task = "task1/"
filename = "pca.csv"
pickle_path = "".join((absPath, "data/results/", task, filename))

#with open(pickle_path, 'wb') as handle:
#    pickle.dump(final_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

final_df.to_csv(pickle_path)

# Task 2

In [ ]:
task = 'task2/'

In [ ]:
# Defining protein to take
idx_2 = random_prot_idx(folder, "task2")
print(idx_2)
#Define list of dataframes
lista_dfs_task2 = []
for model_type in list_paddings:
    print("Model type: ", model_type)
    for fold in range(nfolds):
        model = search_model(folder, str(fold), task, model_type)
        prot, label = load_prot(folder, idx_2, model_type, "labels_task2", nshape)
        activations = obtain_activations(model, nlayer, prot)
        act_df = activations_to_df(activations, model_type, fold, label)
        lista_dfs_task2.append(act_df)

In [ ]:
def saving_activations_plot(packagee, type_plot, p=None):
    if packagee == "plt":
        extension = ".png"
        file_fig = ''.join(string for string in [absPath,'data/results/', folder, task, 'pcaActivations_',
                                             typle_plot, extension])
        plt.savefig(file_fig)
    else:
        extension = ".pdf"
        path_fig = ''.join(string for string in [absPath,'data/results/', folder, task])
        file_fig = ''.join(string for string in ['pcaActivations_', typle_plot, extension])
        p.save(path = path_fig, format = 'pdf', dpi=300, filename=file_fig)

In [ ]:
#tb seria interesante comparar misma conv entre 1conv y rnn